In [28]:
# practice huuuuub so i dont have to create an API call every five minutes LMFAO
import lambda_function as lmbda
import personal_funcs as mf
import boto3
import praw
import json
import pandas as pd
import numpy as np
import datetime as dt
from dateutil import parser
import pprint
from ast import literal_eval
import re
from decimal import Decimal
from functools import reduce
import os
import amazondax
import botocore.session

In [70]:
#! /home/heondokim/envs/reddit_pl/bin/python3
import boto3
import praw
import json
import pandas as pd
import numpy as np
import datetime as dt
from dateutil import parser
import pprint
from ast import literal_eval
import re
from decimal import Decimal
from functools import reduce
import os
import amazondax
import botocore.session


def subs_to_json(top_posts, curr_time, subredd):
    subs_json = []
    for sub in top_posts:
        sub.comments.replace_more(limit=None)
        temp = {"author" : sub.author.name,
        "comments" : sub.comments,
        "date_uploaded": curr_time,
        "created_utc" : sub.created_utc,
        "distinguished" : sub.distinguished,
        "edited" : sub.edited,
        "id" : sub.id,
        "is_self" : sub.is_self,
        # "link_flair_template_id" : sub.link_flair_template_id,
        "link_flair_text" : sub.link_flair_text,
        "locked" : sub.locked,
        "name" : sub.name,
        "num_comments" : sub.num_comments,
        "over_18" : sub.over_18,
        "permalink" : sub.permalink,
        "score" : sub.score,
        "selftext" : sub.selftext,
        "spoiler" : sub.spoiler,
        "stickied" : sub.stickied,
        "subreddit" : subredd,
        "title" : sub.title,
        "upvote_ratio" : sub.upvote_ratio,
        "url" : sub.url
        }
        subs_json.append(temp)
    return subs_json

def commtree_json(comment_tree, curr_time, subredd):
    comm_tree_list = []
    for sub in comment_tree.list():
        try:
            temp = {"author" : sub.author.name,
            "body": sub.body,
            # "comments" : sub.comments,
            "date_uploaded": curr_time,
            "created_utc" : sub.created_utc,
            "distinguished" : sub.distinguished,
            "edited" : sub.edited,
            "id" : sub.id,
            "is_submitter" : sub.is_submitter,
            "link_id" : sub.link_id,
            "parent_id" : sub.parent_id,
            # "locked" : sub.locked,
            # "name" : sub.name,
            # "num_comments" : sub.num_comments,
            # "over_18" : sub.over_18,
            "permalink" : sub.permalink,
            "score" : sub.score,
            # "selftext" : sub.selftext,
            # "spoiler" : sub.spoiler,
            "stickied" : sub.stickied,
            "subreddit_id" : sub.subreddit_id,
            "subreddit" : "weedstocks"
            # "title" : sub.title,
            # "upvote_ratio" : sub.upvote_ratio,
            # "url" : sub.url
            }
            # print(temp)
            comm_tree_list.append(temp)
        except:
            temp = {"author" : None,
            "body": sub.body,
            # "comments" : sub.comments,
            "date_uploaded": curr_time,
            "created_utc" : sub.created_utc,
            "distinguished" : sub.distinguished,
            "edited" : sub.edited,
            "id" : sub.id,
            "is_submitter" : sub.is_submitter,
            "link_id" : sub.link_id,
            "parent_id" : sub.parent_id,
            # "locked" : sub.locked,
            # "name" : sub.name,
            # "num_comments" : sub.num_comments,
            # "over_18" : sub.over_18,
            "permalink" : sub.permalink,
            "score" : sub.score,
            # "selftext" : sub.selftext,
            # "spoiler" : sub.spoiler,
            "stickied" : sub.stickied,
            "subreddit_id" : sub.subreddit_id,
            "subreddit" : "weedstocks"
            # "title" : sub.title,
            # "upvote_ratio" : sub.upvote_ratio,
            # "url" : sub.url
            }
            comm_tree_list.append(temp)
            print(sub.id, "comment deleted")
    return comm_tree_list



def subs_to_comms(jsonofsubs, curr_time, subredd):
    comm_json = []
    for values in jsonofsubs:
        # print(values.items())
        # print(values["comments"])
        temp = commtree_json(values["comments"], curr_time, subredd)
        comm_json.append(temp)
    comm_json1 = [x for x in comm_json if x != []]
    return comm_json1

def remove_comments(subsofjson):
    # what = literal_eval(subs_to_json)
    subsofjson.pop("comments")
    return subsofjson


# build class for praw object...I think
def txt_json(text):
    thing = open(text, 'r').read()
    thing = json.loads(thing, parse_float=Decimal)
    return thing

# i just need...to write a function that will take in the list of submissions
# and create the list of comments. So basically, map comment_tree to sub_json
# what i was doing before is fine i just need to remember to pop the comment forest....
# i believe. but yeah back....again
def upload_to_s3(data, types, time_stamp):
    with open("/tmp/{}_data.txt".format(types), "w") as outfile:
        json.dump(data, outfile)
    s3 = boto3.resource('s3')
    bucket = s3.Bucket('cannabis-stocks')
    bucket.upload_file("/tmp/{}_data.txt".format(types),"{}/{}.txt".format(types, time_stamp))


# functions to take list of json posts, list of list of comments
# convert to dataframes and do cleaning...I think
# def type_conf(list_of_stuff):
#     try:
#         if isinstance(list_of_stuff[0], list):
#             return "comments"
#         else:
#             return "submissions"
#     except:
#         print("neither sub or comment")


def comm_to_df(lists):
    temp = list(map(pd.DataFrame.from_records, lists))
    temp = reduce(lambda x, y: pd.concat([x, y]), temp)
    return temp


def create_df(text_file, typer):
    lists = txt_json('/tmp/{}.txt'.format(text_file))
    if typer == 'comments':
        thing = comm_to_df(lists)
    elif typer == 'submits':
        thing = pd.DataFrame.from_records(lists)
        thing['selftext'] = np.where(thing['selftext'] == "", None, thing['selftext'])
    return thing

def fix_subs(subDF):
    subDF['distinguished'] = subDF['distinguished'].fillna("false")
    subDF['selftext'] = subDF['selftext'].fillna("false")
    # turn it into a json, reread with parse decimal?
    return subDF

def fix_comms(commDF):
    commDF['author'] = commDF['author'].fillna("false")
    commDF['distinguished'] = commDF['distinguished'].fillna("false")
    return commDF

def df_json_dynamo(data, dynadb):
    resource = boto3.resource('dynamodb', region_name='us-west-2')
    table = resource.Table(dynadb)
    item = data.T.to_dict().values()
    for i in item:
        # try:
        table.put_item(Item=i)
        # except:
        #     # so automoderator posts dont work, i think its because the files are different
        #     print('failed with {}'.format(i['id']))

In [24]:
reddit_creds = mf.txt_json('config/reddit_creds.txt')
event = {
        "id": "cdc73f9d-aea9-11e3-9d5a-835b769c0d9c",
        "detail-type": "Scheduled Event",
        "source": "aws.events",
        "account": "{{account-id}}",
        "time": "2019-06-11T00:00:00Z",
        "region": "us-west-2",
        "resources": [
            "arn:aws:events:us-west-2:123456789012:rule/ExampleRule"
        ],
        "detail": {}
        }
context = None
time_rn = parser.parse(event['time'])
time_rn = re.sub(r' ', '_', str(time_rn))

In [25]:
r_object =  lmbda.reddit_posts(configs=reddit_creds, subredd='weedstocks', time_period= 'day', 
                                      limit_num = 5, curr_time=time_rn)
r_object.pop_comments()

etn2wud comment deleted
etm35ot comment deleted
etlg5w3 comment deleted
etlv16k comment deleted
etm5qi0 comment deleted
etl5inu comment deleted
etm845j comment deleted
etlsai3 comment deleted
etm6hv9 comment deleted
etm6tun comment deleted
etm9ejk comment deleted
etmh2fn comment deleted
etlnowp comment deleted
etlvei9 comment deleted
etle91f comment deleted
etlte5u comment deleted
etmddse comment deleted
etlmedh comment deleted
etm8add comment deleted
etm8ial comment deleted
etm5ph9 comment deleted
etmbdve comment deleted
etlf6nh comment deleted
etmawhu comment deleted
etm9ol8 comment deleted
etmaysc comment deleted
etm7pww comment deleted
etm7v2o comment deleted
etm7kto comment deleted
etlnxd0 comment deleted
etlntb6 comment deleted
etloapz comment deleted
etlo1ip comment deleted
etlnwgz comment deleted
etlo01a comment deleted
etleqrh comment deleted
etm686d comment deleted
etm2sld comment deleted
etm5vxq comment deleted
etlfhkm comment deleted
etm2vo6 comment deleted
etmht4p comment 

In [79]:
# now that I know my reddit class works to....get the top posts and submissions.
# I can easily create a class function to send to s3 based on the type of data
# another function for the class is to insert update into dynamoDB, with all of my stuff being cleaned prior.
def fix_subs(subDF):
    subDF['distinguished'] = subDF['distinguished'].fillna("false")
    subDF['selftext'] = subDF['selftext'].fillna("false")
#     sub_j = subDF.to_json(orient='columns')
#     subDF = json.loads(sub_j, parse_float=Decimal)
    return subDF

def fix_comms(commDF):
    commDF['author'] = commDF['author'].fillna("false")
    commDF['distinguished'] = commDF['distinguished'].fillna("false")
#     comm_j = commDF.to_json(orient='columns')
#     commDF = json.loads(comm_j, parse_float=Decimal)
    return commDF

def create_df(text_file):
    lists = txt_json('/tmp/{}_data.txt'.format(text_file))
    if text_file == 'comments':
        thing = comm_to_df(lists)
    elif text_file == 'submits':
        thing = pd.DataFrame.from_records(lists)
        thing['selftext'] = np.where(thing['selftext'] == "", None, thing['selftext'])
    return thing

In [80]:
subDF = create_df("submits")
commDF = create_df("comments")
subDF = fix_subs(subDF)
commDF = fix_comms(commDF)

In [65]:
subDF.head()
print(sorted(commDF.columns))

['author', 'body', 'created_utc', 'date_uploaded', 'distinguished', 'edited', 'id', 'is_submitter', 'link_id', 'parent_id', 'permalink', 'score', 'stickied', 'subreddit', 'subreddit_id']


In [81]:
commDF.head()
# this should be enough for a "cleaned" df that will at least go into dynamodb cleanly....
# anything else?

,author,body,created_utc,date_uploaded,distinguished,edited,id,is_submitter,link_id,parent_id,permalink,score,stickied,subreddit,subreddit_id
0,Gailjnh,"From the article: ""the state legislature has m...",1562928376.0,2019-07-12_17:49:53.711280,false,False,etl9jvx,True,t3_cc99jk,t3_cc99jk,/r/weedstocks/comments/cc99jk/illinois_just_le...,88,False,weedstocks,t5_2zfqj
1,schrikk,I'm all for canabis legalization but no way wo...,1562930646.0,2019-07-12_17:49:53.711280,false,False,etlbeot,False,t3_cc99jk,t3_cc99jk,/r/weedstocks/comments/cc99jk/illinois_just_le...,47,False,weedstocks,t5_2zfqj
2,MaximumRecursion,I know our media and politicians are utter cra...,1562933041.0,2019-07-12_17:49:53.711280,false,False,etldmg4,False,t3_cc99jk,t3_cc99jk,/r/weedstocks/comments/cc99jk/illinois_just_le...,33,False,weedstocks,t5_2zfqj
3,CannaVestments,"""Cassidy stated that the new law as passed wil...",1562936733.0,2019-07-12_17:49:53.711280,false,False,etlhpk9,False,t3_cc99jk,t3_cc99jk,/r/weedstocks/comments/cc99jk/illinois_just_le...,12,False,weedstocks,t5_2zfqj
4,PissedOffPedro,They gotta do something to fix that crippling ...,1562935617.0,2019-07-12_17:49:53.711280,false,False,etlgdy7,False,t3_cc99jk,t3_cc99jk,/r/weedstocks/comments/cc99jk/illinois_just_le...,6,False,weedstocks,t5_2zfqj


In [82]:
def df_json_dynamo(data, dynadb):
    resource = boto3.resource('dynamodb', region_name='us-west-2')
    table = resource.Table(dynadb)
    item = data.T.to_dict().values()
    for i in item:
        table.put_item(Item=i)

In [83]:
df_json_dynamo(subDF, "submits")

/home/heondokim/envs/reddit_pl/lib/python3.5/site-packages/ipykernel_launcher.py:4: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  after removing the cwd from sys.path.
